In [1]:
import spacy
from tqdm.notebook import tqdm
import pickle as pk
from collections import Counter
from nltk.corpus import stopwords
import re
from scipy.special import softmax
import numpy as np
import nltk

In [2]:
# Read CoNLL
result = []
# Read train as well as dev
with open("./eng.train") as ftrain, open("./eng.testa") as fdev:
    document = {}
    lines = []
    for line in ftrain.readlines():
        line = line.split()
        if len(line) > 0:
            if line[1] == '-X-':
                document['conll'] = lines
                document['content'] = ' '.join([line[0] for line in lines])
                result.append(document)
                lines = []
                document = {}
            else:
                lines.append(line)
    for line in fdev.readlines():
        line = line.split()
        if len(line) > 0:
            if line[1] == '-X-':
                document['conll'] = lines
                document['content'] = ' '.join([line[0] for line in lines])
                result.append(document)
                lines = []
                document = {}
            else:
                lines.append(line)

# YAKE!

In [ ]:
import yake

language = "en"
max_key_word_ngram_size = 3
deduplication_thresold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 20
custom_kw_extractor = yake.KeywordExtractor(lan=language,
                                            n=max_key_word_ngram_size, 
                                            dedupLim=deduplication_thresold, 
                                            dedupFunc=deduplication_algo, 
                                            windowsSize=windowSize, 
                                            top=numOfKeywords,
                                            features=None)

for r in tqdm(result):
    keywords = custom_kw_extractor.extract_keywords(r['content'])
    keys = [key for key, value in keywords]
    scores = np.array([value for key, value in keywords])
    # change score from low -> high to high -> low
    scores = 1 - (scores - scores.min()) / (scores.max() - scores.min())
    # Convert list to dict
    keywords = { key : value for key, value in zip(keys, scores)}
    r['keywords'] = keywords

In [124]:
with open("./CoNLL-2003", "wb") as fout:
    pk.dump(result, fout)

# Load keyword extracted CoNLL

In [2]:
with open("./CoNLL-2003", "rb") as fin:
    result = pk.load(fin)

# tf-idf

In [6]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
import numpy as np

vectorizer = TfidfVectorizer(stop_words='english')

corpus = []
for r in result:
    corpus.append(r['content'])
    
X = vectorizer.fit_transform(corpus)
tfidf = X.todense()
# scale score to 0 ~ 1
tfidf = tfidf / tfidf.max(axis=0)
word_array = vectorizer.get_feature_names()
word_dict = vectorizer.vocabulary_
reverse_dict = {v : k for k, v in word_dict.items()}

In [7]:
# Append tf-idf score
for r in tqdm(result):
    for token in r["conll"]:
        if word_dict.get(token[0].lower()) is not None:
            token.append(tfidf[0, word_dict[token[0].lower()]])
        else:
            token.append(0)
    for token in r["conll"]:
        assert len(token) == 5

# YAKE!

In [8]:
max_key_word_ngram_size = 3
# Append yake score
for r in tqdm(result):
    length, token_counter = len(r["conll"]), 0
    keywords = r["keywords"]
    while token_counter < length:
        old_c = token_counter
        # Try to match keyword within keyword ngram
        for key_word_len in range(max_key_word_ngram_size, 0, -1):
            # Find skills
            keyword = " ".join([s[0] for s in r["conll"][token_counter:token_counter+key_word_len]]).lower()
            if (token_counter + key_word_len - 1) < length and keyword in keywords:
                # Tag skills
                for i in range(key_word_len):
                    r['conll'][token_counter + i].append(keywords[keyword])
                # Move pointer
                token_counter += key_word_len
                break
        # No skill is found
        if old_c == token_counter:
            r['conll'][token_counter].append(0)
            token_counter += 1
    for token in r["conll"]:
        assert len(token) == 6

In [9]:
# Load tokenizer
from transformers import BertTokenizer, BertConfig
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
tokenizer.add_special_tokens({"additional_special_tokens" : ["[NOI]", "\n"]})
BertConfig.from_pretrained("bert-base-cased")

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

In [10]:
# Split by BPE
import copy
for r in tqdm(result):
    r["tokenize"] = []
    r["tokenize"].append(('[CLS]', "", "", "start_tkn", 0, 0))
    for t in r["conll"]:
        for i, bpe in enumerate(tokenizer.tokenize(t[0])):
            temp = copy.deepcopy(t)
            # Deal with skill tag
            temp[0] = bpe
            r['tokenize'].append(temp)
    r["tokenize"].append(('[SEP]', "", "", "end_tkn", 0, 0))

# Top down

In [11]:
class Solution:
    def explore(self, index, nums, seen):
        if index >= len(nums):
            return (0, [])

        if index in seen:
            return seen[index]

        # include current house
        val1, path1 = self.explore(index + 2, nums, seen)
        val1 += nums[index]

        # exclude current house
        val2, path2 = self.explore(index + 1, nums, seen)

        if val1 > val2:
            seen[index] = (val1, [index] + path1)
        else:
            seen[index] = (val2, path2)
        return seen[index]

    def rob(self, nums) -> int:
        seen = {}
        val, path = self.explore(0, nums, seen)
        return val, path
s = Solution()

In [12]:
def zero_runs(a):
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 0).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges

# rqm only
training_data = []
noi_ratio = np.zeros(shape=len(result))
for index, r in tqdm(enumerate(result), total=len(result)):
    score_arr = np.zeros(len(r['tokenize']))
    masked_span = np.zeros(len(r['tokenize']))
    if len(r['tokenize']) > 512:
        continue
    for bpe_index, bpe in enumerate(r['tokenize']):
        score, mask_value = 0, 0
        # skill and special token
        if bpe[3] != 'O':
            score = 4
            mask_value = 2
        else:
            # words other than skills and start token
            added = False
            for pos_tag in ["NN", "JJ", "VB"]:
                if pos_tag in bpe[1]:
                    score += 1
                    added = True
                    break
            if not added:
                score += 0.5
            # tf-idf score
            score += bpe[4]
            # yake score
            score += bpe[5]
            mask_value = 0
        # Turn score into negative
        score_arr[bpe_index] = 4-score
        masked_span[bpe_index] = mask_value
    rqm_tkn = [t[0] for t in r['tokenize']]
    training_data.append((rqm_tkn, ["[NOI]"] * len(rqm_tkn)))
    while True:
        zero_ranges = zero_runs(masked_span)
        for span in zero_ranges:
            start, end = span[0], span[1]
            if span[1] - span[0] == 1:
                masked_span[start] = 1
            elif span[1] - span[0] == 2:
                if score_arr[start] > score_arr[end-1]:
                    masked_span[start] = 1
                else:
                    masked_span[end-1] = 1
            else:
                value, path = s.rob(score_arr[start:end])
                for p in path:
                    masked_span[start+p] = 1
        if len(zero_ranges) == 0:
            break
        train, label = [], []
        select_cursor = 0
        for i in range(len(masked_span)):
            if masked_span[i] != 1:
                train.append(rqm_tkn[i])
                if i + 1 < len(masked_span) and masked_span[i + 1] == 1:
                    label.append(rqm_tkn[i+1])
                    select_cursor += 1
                else:
                    label.append("[NOI]")
        training_data.append((train, label))
        rqm_tkn = train
        score_arr = score_arr[masked_span != 1]
        masked_span = masked_span[masked_span != 1]
        assert len(masked_span) == len(rqm_tkn) == len(score_arr)
    noi_ratio[index] = Counter(training_data[-1][1])['[NOI]'] / len(training_data[-1][1])
    
with open("CoNLL-2003_InsertionTransformer", "wb") as fout:
    pk.dump(training_data, fout)

In [14]:
noi_ratio.mean(), len(training_data)

(0.8206064121540154, 6557)

# TD example

In [16]:
def zero_runs(a):
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 0).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges

# rqm only
example_arr = []
noi_ratio = np.zeros(shape=len(result))
for index, r in tqdm(enumerate(result), total=len(result)):
    training_data = []
    score_arr = np.zeros(len(r['tokenize']))
    masked_span = np.zeros(len(r['tokenize']))
    if len(r['tokenize']) > 512:
        continue
    for bpe_index, bpe in enumerate(r['tokenize']):
        score, mask_value = 0, 0
        # skill and special token
        if bpe[3] != 'O':
            score = 4
            mask_value = 2
        else:
            # words other than skills and start token
            added = False
            for pos_tag in ["NN", "JJ", "VB"]:
                if pos_tag in bpe[1]:
                    score += 1
                    added = True
                    break
            if not added:
                score += 0.5
            # tf-idf score
            score += bpe[4]
            # yake score
            score += bpe[5]
            mask_value = 0
        # Turn score into negative
        score_arr[bpe_index] = 4-score
        masked_span[bpe_index] = mask_value
    rqm_tkn = [t[0] for t in r['tokenize']]
    training_data.append((rqm_tkn, ["[NOI]"] * len(rqm_tkn)))
    while True:
        zero_ranges = zero_runs(masked_span)
        for span in zero_ranges:
            start, end = span[0], span[1]
            if span[1] - span[0] == 1:
                masked_span[start] = 1
            elif span[1] - span[0] == 2:
                if score_arr[start] > score_arr[end-1]:
                    masked_span[start] = 1
                else:
                    masked_span[end-1] = 1
            else:
                value, path = s.rob(score_arr[start:end])
                for p in path:
                    masked_span[start+p] = 1
        if len(zero_ranges) == 0:
            break
        train, label = [], []
        select_cursor = 0
        for i in range(len(masked_span)):
            if masked_span[i] != 1:
                train.append(rqm_tkn[i])
                if i + 1 < len(masked_span) and masked_span[i + 1] == 1:
                    label.append(rqm_tkn[i+1])
                    select_cursor += 1
                else:
                    label.append("[NOI]")
        training_data.append((train, label))
        rqm_tkn = train
        score_arr = score_arr[masked_span != 1]
        masked_span = masked_span[masked_span != 1]
        assert len(masked_span) == len(rqm_tkn) == len(score_arr)
    noi_ratio[index] = Counter(training_data[-1][1])['[NOI]'] / len(training_data[-1][1])
    example_arr.append(training_data)
    
with open("CoNLL-2003_InsertionTransformer_example", "wb") as fout:
    pk.dump(example_arr, fout)

In [17]:
len(example_arr)

1004

# Top down original

In [58]:
noi_ratio = np.zeros(shape=len(result))
training_data = []
for index, r in tqdm(enumerate(result), total=len(result)):
    score_arr = []
    for bpe in r['tokenize']:
        score = 0
        # POS tag score
        # entity token
        if bpe[3] != 'O':
            score = 4
        else:
            # words other than skills and start token
            added = False
            for pos_tag in ["NN", "JJ", "VB"]:
                if pos_tag in bpe[1]:
                    score += 1
                    added = True
                    break
            if not added:
                score += 0.5
            # tf-idf score
            score += bpe[4]
            # yake score
            score += bpe[5]
        # Turn score into negative
        score_arr.append(-score)
    tkns = [t[0] for t in r['tokenize']]
    if len(tkns) > 512:
        continue
    training_data.append((tkns, ["[NOI]"] * len(tkns)))
    loop = True
    while loop:
        mask = [1] * len(score_arr)
        for i in range(len(score_arr)):
            max_value = float('-inf')
            for j in range(len(score_arr)):
                if score_arr[j] > max_value and mask[j] == 1 and (j == 0 or mask[j - 1] == 1) and (j == len(score_arr) - 1 or mask[j + 1] == 1):
                    max_value = score_arr[j]
                    max_index = j
            # If the token chosen is skill
            if max_value == -4:
                break
            # Mask selected index
            mask[max_index] = 0
        # If all token remains
        if all(mask):
            loop = False
            break
        else:
            train, label, new_score = [], [], []
            for i, m in enumerate(mask):
                if m == 1:
                    train.append(tkns[i])
                    new_score.append(score_arr[i])
                    if i + 1 < len(mask) and mask[i + 1] == 0:
                        label.append(tkns[i + 1])
                    else:
                        label.append("[NOI]")
            tkns = train
            score_arr = new_score
            training_data.append((train, label))
    noi_ratio[index] = Counter(training_data[-1][1])['[NOI]'] / len(training_data[-1][1])
    
with open("CoNLL-2003_InsertionTransformer", "wb") as fout:
    pk.dump(training_data, fout)

In [59]:
noi_ratio.mean()

0.8273283202800721

# POINTER

In [27]:
def zero_runs(a):
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 0).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges

# rqm only
training_data = []
noi_ratio = np.zeros(shape=len(result))
for index, r in tqdm(enumerate(result), total=len(result)):
    score_arr = np.zeros(len(r['tokenize']))
    masked_span = np.zeros(len(r['tokenize']))
    masked_span[0], masked_span[-1] = 2, 2
    if len(r['tokenize']) > 512:
        continue
    for bpe_index, bpe in enumerate(r['tokenize']):
        score = 0
        # skill and special token
        # words other than skills and start token
        added = False
        for pos_tag in ["NN", "JJ", "VB"]:
            if pos_tag in bpe[1]:
                score += 1
                added = True
                break
        if not added:
            score += 0.5
        # tf-idf score
        score += bpe[4]
        # yake score
        score += bpe[5]
        # Turn score into negative
        score_arr[bpe_index] = 4-score
    rqm_tkn = [t[0] for t in r['tokenize']]
    training_data.append((rqm_tkn, ["[NOI]"] * len(rqm_tkn)))
    initial_length = len(r['tokenize'])
    while len(rqm_tkn) > initial_length // 7:
        zero_ranges = zero_runs(masked_span)
        for span in zero_ranges:
            start, end = span[0], span[1]
            if span[1] - span[0] == 1:
                masked_span[start] = 1
            elif span[1] - span[0] == 2:
                if score_arr[start] > score_arr[end-1]:
                    masked_span[start] = 1
                else:
                    masked_span[end-1] = 1
            else:
                value, path = s.rob(score_arr[start:end])
                for p in path:
                    masked_span[start+p] = 1
        if len(zero_ranges) == 0:
            break
        train, label = [], []
        select_cursor = 0
        for i in range(len(masked_span)):
            if masked_span[i] != 1:
                train.append(rqm_tkn[i])
                if i + 1 < len(masked_span) and masked_span[i + 1] == 1:
                    label.append(rqm_tkn[i+1])
                    select_cursor += 1
                else:
                    label.append("[NOI]")
        training_data.append((train, label))
        rqm_tkn = train
        score_arr = score_arr[masked_span != 1]
        masked_span = masked_span[masked_span != 1]
        assert len(masked_span) == len(rqm_tkn) == len(score_arr)
    noi_ratio[index] = Counter(training_data[-1][1])['[NOI]'] / len(training_data[-1][1])
    
with open("CoNLL-2003_InsertionTransformer_pointer", "wb") as fout:
    pk.dump(training_data, fout)

In [ ]:
result[0]

# POINTER enum

In [32]:
def zero_runs(a):
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 0).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges

# rqm only
training_data = []
noi_ratio = np.zeros(shape=len(result))
for index, r in tqdm(enumerate(result), total=len(result)):
    score_arr = np.zeros(len(r['tokenize']))
    masked_span = np.zeros(len(r['tokenize']))
    masked_span[0], masked_span[-1] = 2, 2
    if len(r['tokenize']) > 512:
        continue
    for bpe_index, bpe in enumerate(r['tokenize']):
        score = 0
        # skill and special token
        # words other than skills and start token
        added = False
        for pos_tag in ["NN", "JJ", "VB"]:
            if pos_tag in bpe[1]:
                score += 1
                added = True
                break
        if not added:
            score += 0.5
        # tf-idf score
        score += bpe[4]
        # yake score
        score += bpe[5]
        # Turn score into negative
        score_arr[bpe_index] = 4-score
    rqm_tkn = [t[0] for t in r['tokenize']]
    training_data.append((rqm_tkn, ["[NOI]"] * len(rqm_tkn)))
    initial_length = 0
    for skl in r['entities']:
        initial_length += len(tokenizer.tokenize(' '.join(skl)))
    while len(rqm_tkn) > initial_length:
        zero_ranges = zero_runs(masked_span)
        for span in zero_ranges:
            start, end = span[0], span[1]
            if span[1] - span[0] == 1:
                masked_span[start] = 1
            elif span[1] - span[0] == 2:
                if score_arr[start] > score_arr[end-1]:
                    masked_span[start] = 1
                else:
                    masked_span[end-1] = 1
            else:
                value, path = s.rob(score_arr[start:end])
                for p in path:
                    masked_span[start+p] = 1
        if len(zero_ranges) == 0:
            break
        train, label = [], []
        select_cursor = 0
        for i in range(len(masked_span)):
            if masked_span[i] != 1:
                train.append(rqm_tkn[i])
                if i + 1 < len(masked_span) and masked_span[i + 1] == 1:
                    label.append(rqm_tkn[i+1])
                    select_cursor += 1
                else:
                    label.append("[NOI]")e
        training_data.append((train, label))
        rqm_tkn = train
        score_arr = score_arr[masked_span != 1]
        masked_span = masked_span[masked_span != 1]
        assert len(masked_span) == len(rqm_tkn) == len(score_arr)
    noi_ratio[index] = Counter(training_data[-1][1])['[NOI]'] / len(training_data[-1][1])
    
with open("CoNLL-2003_InsertionTransformer_pointer_enum", "wb") as fout:
    pk.dump(training_data, fout)

# Bottom up wired algo

In [ ]:
# Deprecated because wired algo setting

# rqm only
training_data = []
noi_ratio = np.zeros(shape=len(result))
for index, r in tqdm(enumerate(result), total=len(result)):
    score_arr = []
    masked_span = []
    if len(r['tokenize']) > 512:
        continue
    for bpe in r['tokenize']:
        score, mask_value = 0, 0
        # POS tag score
        # Skill and special token
        if bpe[3] != 'O':
            score = 4
            mask_value = 1
        else:
            # words other than skills and start token
            added = False
            for pos_tag in ["NN", "JJ", "VB"]:
                if pos_tag in bpe[1]:
                    score += 1
                    added = True
                    break
            if not added:
                score += 0.5
            # tf-idf score
            score += bpe[4]
            # yake score
            score += bpe[5]
            mask_value = 0
        # Turn score into negative
        score_arr.append(score)
        masked_span.append(mask_value)
    tkns = [t[0] for t in r['tokenize']]
    # Record number of NOI
    recorded = False
    while not all(masked_span):
        cursor = 0
        start, end = None, None
        max_reward, max_reward_idx = float('-inf'), None
        insert_index = []
        while cursor < len(masked_span):
            if masked_span[cursor] == 0:
                if start is None:
                    start = cursor
                    end = cursor
                else:
                    end = cursor
                    if (end - start > 0 and (end + 1) < len(masked_span) and masked_span[end + 1] != 1 
                        and score_arr[cursor] > max_reward):
                        max_reward = score_arr[cursor]
                        max_reward_idx = cursor
            elif end is not None:
                if end - start + 1 > 2:
                    insert_index.append(max_reward_idx)
                else:
                    if score_arr[start] > score_arr[end]:
                        insert_index.append(start)
                    else:
                        insert_index.append(end)
                # Clear span
                start, end = None, None
                max_reward, max_reward_idx = float('-inf'), None
            cursor += 1
        train, label = [], []
        select_cursor = 0
        for i, m, r in zip(range(len(masked_span)), masked_span, tkns):
            if m == 1:
                train.append(r)
                if i + 1 < len(masked_span) and masked_span[i + 1] == 0:
                    label.append(tkns[insert_index[select_cursor]])
                    select_cursor += 1
                else:
                    label.append("[NOI]")
        training_data.append((train, label))
        if not recorded:
            noi_ratio[index] = Counter(training_data[-1][1])['[NOI]'] / len(training_data[-1][1])
            recorded = True
        for i_idx in insert_index:
            masked_span[i_idx] = 1
    training_data.append((tkns, ["[NOI]"] * len(tkns)))

with open("CoNLL-2003_InsertionTransformer_bottom_up", "wb") as fout:
    pk.dump(training_data, fout)

# Bottom up softmax

In [ ]:
# Deprecared
# Use softmax as one of the scores

def generate_distance(start, end):
    left_bound, right_bound = start - 1, end + 1
    distance = [min(i - left_bound, right_bound - i) for i in range(start, end+1)]
    return distance

# rqm only
training_data = []
for r in tqdm(result):
    score_arr = []
    masked_span = []
    if len(r['tokenize']) > 512:
        continue
    for bpe in r['tokenize']:
        score, mask_value = 0, 0
        # POS tag score
        # Skill and special token
        if bpe[3] != 'O':
            score = 4
            mask_value = 1
        else:
            # words other than skills and start token
            added = False
            for pos_tag in ["NN", "JJ", "VB"]:
                if pos_tag in bpe[1]:
                    score += 1
                    added = True
                    break
            if not added:
                score += 0.5
            # tf-idf score
            score += bpe[4]
            # yake score
            score += bpe[5]
            mask_value = 0
        # Turn score into negative
        score_arr.append(score)
        masked_span.append(mask_value)
    score_arr = np.array(score_arr)
    tkns = [t[0] for t in r['tokenize']]
    while not all(masked_span):
        cursor = 0
        start, end = None, None
        max_reward, max_reward_idx = float('-inf'), None
        insert_index = []
        while cursor < len(masked_span):
            if masked_span[cursor] == 0:
                if start is None:
                    start = cursor
                    end = cursor
                else:
                    end = cursor
            elif end is not None:
                overall_score = score_arr[start:end+1]
                softmax_score = softmax(generate_distance(start, end))
                if softmax_score.max() - softmax_score.min() == 0:
                    overall_score += softmax_score
                else:
                    overall_score += (softmax_score - softmax_score.min()) / (softmax_score.max() - softmax_score.min())
                insert_index.append(start + overall_score.argmax())
                # Clear span
                start, end = None, None
            cursor += 1
        train, label = [], []
        select_cursor = 0
        for i, m, r in zip(range(len(masked_span)), masked_span, tkns):
            if m == 1:
                train.append(r)
                if i + 1 < len(masked_span) and masked_span[i + 1] == 0:
                    label.append(tkns[insert_index[select_cursor]])
                    select_cursor += 1
                else:
                    label.append("[NOI]")
        training_data.append((train, label))
        for i_idx in insert_index:
            masked_span[i_idx] = 1
    training_data.append((tkns, ["[NOI]"] * len(tkns)))
    
with open("CoNLL-2003_InsertionTransformer_bottom_up_softmax", "wb") as fout:
    pk.dump(training_data, fout)

In [17]:
# Use softmax as masks
def generate_distance(start, end):
    left_bound, right_bound = start - 1, end + 1
    distance = [min(i - left_bound, right_bound - i) for i in range(start, end+1)]
    return distance

# rqm only
# Use softmax as a mask (multiply)
training_data = []
for r in tqdm(result):
    score_arr = []
    masked_span = []
    if len(r['tokenize']) > 512:
        continue
    for bpe in r['tokenize']:
        score, mask_value = 0, 0
        # POS tag score
        # Skill and special token
        if bpe[3] != 'O':
            score = 4
            mask_value = 1
        else:
            # words other than skills and start token
            added = False
            for pos_tag in ["NN", "JJ", "VB"]:
                if pos_tag in bpe[1]:
                    score += 1
                    added = True
                    break
            if not added:
                score += 0.5
            # tf-idf score
            score += bpe[4]
            # yake score
            score += bpe[5]
            mask_value = 0
        # Turn score into negative
        score_arr.append(score)
        masked_span.append(mask_value)
    score_arr = np.array(score_arr)
    tkns = [t[0] for t in r['tokenize']]
    while not all(masked_span):
        cursor = 0
        start, end = None, None
        max_reward, max_reward_idx = float('-inf'), None
        insert_index = []
        while cursor < len(masked_span):
            if masked_span[cursor] == 0:
                if start is None:
                    start = cursor
                    end = cursor
                else:
                    end = cursor
            elif end is not None:
                overall_score = score_arr[start:end+1]
                softmax_score = softmax(generate_distance(start, end))
                if softmax_score.max() - softmax_score.min() != 0:
                    overall_score *= (softmax_score - softmax_score.min()) / (softmax_score.max() - softmax_score.min())
                insert_index.append(start + overall_score.argmax())
                # Clear span
                start, end = None, None
            cursor += 1
        train, label = [], []
        select_cursor = 0
        for i, m, r in zip(range(len(masked_span)), masked_span, tkns):
            if m == 1:
                train.append(r)
                if i + 1 < len(masked_span) and masked_span[i + 1] == 0:
                    label.append(tkns[insert_index[select_cursor]])
                    select_cursor += 1
                else:
                    label.append("[NOI]")
        training_data.append((train, label))
        for i_idx in insert_index:
            masked_span[i_idx] = 1
    training_data.append((tkns, ["[NOI]"] * len(tkns)))
    
with open("CoNLL-2003_InsertionTransformer_bottom_up_softmax_multiply", "wb") as fout:
    pk.dump(training_data, fout)

In [18]:
len(training_data)

8492

# Bottom up softmax example

In [21]:
def generate_distance(start, end):
    left_bound, right_bound = start - 1, end + 1
    distance = [min(i - left_bound, right_bound - i) for i in range(start, end+1)]
    return distance

# rqm only
# Use softmax as a mask (multiply)
example_arr = []
for r in tqdm(result):
    training_data = []
    score_arr = []
    masked_span = []
    if len(r['tokenize']) > 512:
        continue
    for bpe in r['tokenize']:
        score, mask_value = 0, 0
        # POS tag score
        # Skill and special token
        if bpe[3] != 'O':
            score = 4
            mask_value = 1
        else:
            # words other than skills and start token
            added = False
            for pos_tag in ["NN", "JJ", "VB"]:
                if pos_tag in bpe[1]:
                    score += 1
                    added = True
                    break
            if not added:
                score += 0.5
            # tf-idf score
            score += bpe[4]
            # yake score
            score += bpe[5]
            mask_value = 0
        # Turn score into negative
        score_arr.append(score)
        masked_span.append(mask_value)
    score_arr = np.array(score_arr)
    tkns = [t[0] for t in r['tokenize']]
    while not all(masked_span):
        cursor = 0
        start, end = None, None
        max_reward, max_reward_idx = float('-inf'), None
        insert_index = []
        while cursor < len(masked_span):
            if masked_span[cursor] == 0:
                if start is None:
                    start = cursor
                    end = cursor
                else:
                    end = cursor
            elif end is not None:
                overall_score = score_arr[start:end+1]
                softmax_score = softmax(generate_distance(start, end))
                if softmax_score.max() - softmax_score.min() != 0:
                    overall_score *= (softmax_score - softmax_score.min()) / (softmax_score.max() - softmax_score.min())
                insert_index.append(start + overall_score.argmax())
                # Clear span
                start, end = None, None
            cursor += 1
        train, label = [], []
        select_cursor = 0
        for i, m, r in zip(range(len(masked_span)), masked_span, tkns):
            if m == 1:
                train.append(r)
                if i + 1 < len(masked_span) and masked_span[i + 1] == 0:
                    label.append(tkns[insert_index[select_cursor]])
                    select_cursor += 1
                else:
                    label.append("[NOI]")
        training_data.append((train, label))
        for i_idx in insert_index:
            masked_span[i_idx] = 1
    training_data.append((tkns, ["[NOI]"] * len(tkns)))
    example_arr.append(training_data)
    
with open("CoNLL-2003_InsertionTransformer_bottom_up_softmax_multiply_example", "wb") as fout:
    pk.dump(example_arr, fout)

# Bottom up GGD

In [19]:
from scipy.stats import gennorm

# Use ggd as one of the scores

beta = 8

# rqm only
training_data = []
for r in tqdm(result):
    score_arr = []
    masked_span = []
    if len(r['tokenize']) > 512:
        continue
    for bpe in r['tokenize']:
        score, mask_value = 0, 0
        # POS tag score
        # Skill and special token
        if bpe[3] != 'O':
            score = 4
            mask_value = 1
        else:
            # words other than skills and start token
            added = False
            for pos_tag in ["NN", "JJ", "VB"]:
                if pos_tag in bpe[1]:
                    score += 1
                    added = True
                    break
            if not added:
                score += 0.5
            # tf-idf score
            score += bpe[4]
            # yake score
            score += bpe[5]
            mask_value = 0
        # Turn score into negative
        score_arr.append(score)
        masked_span.append(mask_value)
    score_arr = np.array(score_arr)
    tkns = [t[0] for t in r['tokenize']]
    while not all(masked_span):
        cursor = 0
        start, end = None, None
        max_reward, max_reward_idx = float('-inf'), None
        insert_index = []
        while cursor < len(masked_span):
            if masked_span[cursor] == 0:
                if start is None:
                    start = cursor
                    end = cursor
                else:
                    end = cursor
            elif end is not None:
                overall_score = score_arr[start:end+1]
                ggd_score = gennorm.pdf(np.linspace(gennorm.ppf(0.01, beta),
                                                    gennorm.ppf(0.99, beta), end - start + 1),
                                        beta)
                if ggd_score.max() - ggd_score.min() == 0:
                    overall_score += np.ones(shape=ggd_score.shape)
                else:
                    overall_score += (ggd_score - ggd_score.min()) / (ggd_score.max() - ggd_score.min())
                insert_index.append(start + overall_score.argmax())
                # Clear span
                start, end = None, None
            cursor += 1
        train, label = [], []
        select_cursor = 0
        for i, m, r in zip(range(len(masked_span)), masked_span, tkns):
            if m == 1:
                train.append(r)
                if i + 1 < len(masked_span) and masked_span[i + 1] == 0:
                    label.append(tkns[insert_index[select_cursor]])
                    select_cursor += 1
                else:
                    label.append("[NOI]")
        training_data.append((train, label))
        for i_idx in insert_index:
            masked_span[i_idx] = 1
    training_data.append((tkns, ["[NOI]"] * len(tkns)))
    
with open("CoNLL-2003_InsertionTransformer_bottom_up_ggd", "wb") as fout:
    pk.dump(training_data, fout)

In [ ]:
from scipy.stats import gennorm


# Use GGD as masks
beta = 8

# rqm only
training_data = []
for r in tqdm(result):
    score_arr = []
    masked_span = []
    if len(r['tokenize']) > 512:
        continue
    for bpe in r['tokenize']:
        score, mask_value = 0, 0
        # POS tag score
        # Skill and special token
        if bpe[3] != 'O':
            score = 4
            mask_value = 1
        else:
            # words other than skills and start token
            added = False
            for pos_tag in ["NN", "JJ", "VB"]:
                if pos_tag in bpe[1]:
                    score += 1
                    added = True
                    break
            if not added:
                score += 0.5
            # tf-idf score
            score += bpe[4]
            # yake score
            score += bpe[5]
            mask_value = 0
        # Turn score into negative
        score_arr.append(score)
        masked_span.append(mask_value)
    score_arr = np.array(score_arr)
    tkns = [t[0] for t in r['tokenize']]
    while not all(masked_span):
        cursor = 0
        start, end = None, None
        max_reward, max_reward_idx = float('-inf'), None
        insert_index = []
        while cursor < len(masked_span):
            if masked_span[cursor] == 0:
                if start is None:
                    start = cursor
                    end = cursor
                else:
                    end = cursor
            elif end is not None:
                overall_score = score_arr[start:end+1]
                ggd_score = gennorm.pdf(np.linspace(gennorm.ppf(0.01, beta),
                                                    gennorm.ppf(0.99, beta), end - start + 1),
                                        beta)
                if ggd_score.max() - ggd_score.min() != 0:
                    overall_score *= (ggd_score - ggd_score.min()) / (ggd_score.max() - ggd_score.min())
                insert_index.append(start + overall_score.argmax())
                # Clear span
                start, end = None, None
            cursor += 1
        train, label = [], []
        select_cursor = 0
        for i, m, r in zip(range(len(masked_span)), masked_span, tkns):
            if m == 1:
                train.append(r)
                if i + 1 < len(masked_span) and masked_span[i + 1] == 0:
                    label.append(tkns[insert_index[select_cursor]])
                    select_cursor += 1
                else:
                    label.append("[NOI]")
        training_data.append((train, label))
        for i_idx in insert_index:
            masked_span[i_idx] = 1
    training_data.append((tkns, ["[NOI]"] * len(tkns)))
    
with open("CoNLL-2003_InsertionTransformer_bottom_up_ggd_multiply", "wb") as fout:
    pk.dump(training_data, fout)

# Bottom up only

In [21]:
# rqm only
# Use softmax as a mask (multiply)
training_data = []
for r in tqdm(result):
    score_arr = []
    masked_span = []
    if len(r['tokenize']) > 512:
        continue
    for bpe in r['tokenize']:
        score, mask_value = 0, 0
        # POS tag score
        # Skill and special token
        if bpe[3] != 'O':
            score = 4
            mask_value = 1
        else:
            # words other than skills and start token
            added = False
            for pos_tag in ["NN", "JJ", "VB"]:
                if pos_tag in bpe[1]:
                    score += 1
                    added = True
                    break
            if not added:
                score += 0.5
            # tf-idf score
            score += bpe[4]
            # yake score
            score += bpe[5]
            mask_value = 0
        # Turn score into negative
        score_arr.append(score)
        masked_span.append(mask_value)
    score_arr = np.array(score_arr)
    tkns = [t[0] for t in r['tokenize']]
    while not all(masked_span):
        cursor = 0
        start, end = None, None
        max_reward, max_reward_idx = float('-inf'), None
        insert_index = []
        while cursor < len(masked_span):
            if masked_span[cursor] == 0:
                if start is None:
                    start = cursor
                    end = cursor
                else:
                    end = cursor
            elif end is not None:
                overall_score = score_arr[start:end+1]
                insert_index.append(start + overall_score.argmax())
                # Clear span
                start, end = None, None
            cursor += 1
        train, label = [], []
        select_cursor = 0
        for i, m, r in zip(range(len(masked_span)), masked_span, tkns):
            if m == 1:
                train.append(r)
                if i + 1 < len(masked_span) and masked_span[i + 1] == 0:
                    label.append(tkns[insert_index[select_cursor]])
                    select_cursor += 1
                else:
                    label.append("[NOI]")
        training_data.append((train, label))
        for i_idx in insert_index:
            masked_span[i_idx] = 1
    training_data.append((tkns, ["[NOI]"] * len(tkns)))
    
with open("CoNLL-2003_InsertionTransformer_bottom_up_only", "wb") as fout:
    pk.dump(training_data, fout)

In [22]:
noi_ratio.mean()
len(training_data)

17694

# Bottom up example

In [18]:
# rqm only
# Use softmax as a mask (multiply)
example_arr = []
for r in tqdm(result):
    training_data = []
    score_arr = []
    masked_span = []
    if len(r['tokenize']) > 512:
        continue
    for bpe in r['tokenize']:
        score, mask_value = 0, 0
        # POS tag score
        # Skill and special token
        if bpe[3] != 'O':
            score = 4
            mask_value = 1
        else:
            # words other than skills and start token
            added = False
            for pos_tag in ["NN", "JJ", "VB"]:
                if pos_tag in bpe[1]:
                    score += 1
                    added = True
                    break
            if not added:
                score += 0.5
            # tf-idf score
            score += bpe[4]
            # yake score
            score += bpe[5]
            mask_value = 0
        # Turn score into negative
        score_arr.append(score)
        masked_span.append(mask_value)
    score_arr = np.array(score_arr)
    tkns = [t[0] for t in r['tokenize']]
    while not all(masked_span):
        cursor = 0
        start, end = None, None
        max_reward, max_reward_idx = float('-inf'), None
        insert_index = []
        while cursor < len(masked_span):
            if masked_span[cursor] == 0:
                if start is None:
                    start = cursor
                    end = cursor
                else:
                    end = cursor
            elif end is not None:
                overall_score = score_arr[start:end+1]
                insert_index.append(start + overall_score.argmax())
                # Clear span
                start, end = None, None
            cursor += 1
        train, label = [], []
        select_cursor = 0
        for i, m, r in zip(range(len(masked_span)), masked_span, tkns):
            if m == 1:
                train.append(r)
                if i + 1 < len(masked_span) and masked_span[i + 1] == 0:
                    label.append(tkns[insert_index[select_cursor]])
                    select_cursor += 1
                else:
                    label.append("[NOI]")
        training_data.append((train, label))
        for i_idx in insert_index:
            masked_span[i_idx] = 1
    training_data.append((tkns, ["[NOI]"] * len(tkns)))
    example_arr.append(training_data)
    
with open("CoNLL-2003_InsertionTransformer_bottom_up_only_example", "wb") as fout:
    pk.dump(example_arr, fout)

# Testing data

In [5]:
#Prepare testing data for requirement insertion only

with open("eng.testb") as fin:
    testing, lines = [], []
    for line in fin.readlines():
        line = line.split()
        if len(line) > 0:
            if line[1] == '-X-':
                testing.append(lines)
                lines = []
            else:
                lines.append(line)

testing_data = []
for test in tqdm(testing):
    gt = ' '.join([line[0] for line in test])
    content = ' '.join([line[0] for line in test if line[-1] != 'O'])
    testing_data.append((" [CLS] " + content + " [SEP] ", gt))
    
with open("CoNLL-2003_InsertionTransformer_test", "wb") as fout:
    pk.dump(testing_data, fout)

In [5]:
#Prepare testing data for requirement only but with span inference the phrase

with open("eng.testb") as fin:
    testing, lines = [], []
    for line in fin.readlines():
        line = line.split()
        if len(line) > 0:
            if line[1] == '-X-':
                testing.append(lines)
                lines = []
            else:
                lines.append(line)
                
entities = []
for test in tqdm(testing):
    entity = []
    tmp_arr = []
    last_tag = ''
    for line in test:
        if len(tmp_arr) != 0 and last_tag != line[3]:
            entity.append(tmp_arr)
            tmp_arr = []
        if line[3] != 'O':
            tmp_arr.append(line[0])
        last_tag = line[3]
    entities.append(entity)

testing_data = []
for test, entity in tqdm(zip(testing, entities), total=len(testing)):
    gt = ' '.join([line[0] for line in test])
    out_str = []
    phrase_num = []
    for i, e in enumerate(entity):
        tokens = tokenizer.tokenize(' '.join(e))
        out_str.append(' '.join(e))
        phrase_num += [i] * len(tokens)
    # add begin and end token 
    phrase_num = [-1] + phrase_num + [-1]
    testing_data.append((" [CLS] " + " ".join(out_str) + " [SEP] ", gt, phrase_num))
    
with open("CoNLL-2003_InsertionTransformer_test_span_inf", "wb") as fout:
    pk.dump(testing_data, fout)